<a href="https://colab.research.google.com/github/YuerTang/CS88/blob/main/188_ca2_kickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make a copy of this!

In [ ]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import os

from IPython.display import display, Image
from matplotlib import patches
%matplotlib inline

Run these cells for some useful helper functions

In [ ]:
def show_image(img, title="Image", cmap=None):
    plt.figure(figsize=(10, 8))
    if cmap is None and len(img.shape) == 3:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
    else:
        plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

Upload an image we will use (run the cell). You should be using an image with something red in it. We have some defaults you can download here: https://drive.google.com/drive/folders/1l2kCb40Wf7DVoQdPHKzsXzBTYmZJlTO7?usp=sharing

I personally used ca2_0.png.

In [ ]:
uploader = widgets.FileUpload(
    accept='.jpg, .jpeg, .png',
    multiple=False,
    description='Upload Image'
)

def load_image(change=None):
    if uploader.value and len(uploader.value) > 0:
        file_info = next(iter(uploader.value.items()))
        file_name, file_data = file_info

        img_array = np.frombuffer(file_data['content'], dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        print(f"Using uploaded image: {file_name}")
        return img
    else:
        print("Failed to upload image.")

display(uploader)

process_button = widgets.Button(description="Process Image")
display(process_button)

def on_process_button_clicked(b):
    original_image = load_image()
    if original_image is not None:
        cv2.imwrite('temp_display.jpg', original_image)
        display(Image('temp_display.jpg'))
        print("Image loaded successfully! Shape:", original_image.shape)
    else:
        print("Failed to load image.")

process_button.on_click(on_process_button_clicked)


FileUpload(value={}, accept='.jpg, .jpeg, .png', description='Upload Image')

Button(description='Process Image', style=ButtonStyle())

Load our image and show it!

In [ ]:
original_image = load_image()
show_image(original_image)

Failed to upload image.


AttributeError: 'NoneType' object has no attribute 'shape'

<Figure size 1000x800 with 0 Axes>

Please implement this function. OpenCV uses BGR (Blue, Green, Red) by default. For the purposes of detection, we will be using HSV (Hue, Saturation, Value). Why might we want to do things in HSV space?

Hint for implementation: look into cv2.cvtColor

In [ ]:
def bgr_to_hsv(input_image):
    """
    Convert an RGB image to HSV color space.

    Parameters:
        input_image (np.ndarray): Input RGB image (BGR format as used by OpenCV).

    Returns:
        np.ndarray: HSV image.
    """
    pass

Use this function to display the different channels (hue, saturation, value). (Nothing to implement here).

In [ ]:
hsv_image = bgr_to_hsv(original_image)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(hsv_image[:,:,0], cmap='hsv')
axes[0].set_title('Hue Channel')
axes[0].axis('off')

axes[1].imshow(hsv_image[:,:,1], cmap='gray')
axes[1].set_title('Saturation Channel')
axes[1].axis('off')

axes[2].imshow(hsv_image[:,:,2], cmap='gray')
axes[2].set_title('Value Channel')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In order to detect red in the image, we will create masks for red color in the HSV image. Two guiding questions:

1. What channel should we be masking over?

2. How many masks will we need? Why?

Hint 1: One way to approach this is to implement a lower and upper mask to define a range. You could write something like mask = np.array([hue, sat, val]) for a single mask / bound.

Hint 2: From your answer in (1), you will need to play around with some minimum and maximum values for the other channels as well.

Hint 3: Once you have your upper and lower masks (i.e. you defined your range of values), look into cv2.inRange.

In [ ]:
def create_red_masks(hsv_image):
    """
    Create a mask for red color in the HSV image.

    Parameters:
        hsv_image (np.ndarray): Input HSV image.

    Returns:
        np.ndarray: Binary mask where red pixels are white and others are black.
    """
    pass

Visualize the two masks. Note that the upper red mask will not display much... why? (Nothing to implement here).

In [ ]:
mask1, mask2 = create_red_masks(hsv_image)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(mask1, cmap='gray')
axes[0].set_title('Mask 1 (Lower Red)')
axes[0].axis('off')

axes[1].imshow(mask2, cmap='gray')
axes[1].set_title('Mask 2 (Upper Red)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

From earlier, we have two separate masks to mask out the red from the image. Which binary operation can we use to combine them? Implement!

In [ ]:
def combine_masks(mask1, mask2):
    """
    Combine two binary masks.

    Parameters:
        mask1 (np.ndarray): First binary mask.
        mask2 (np.ndarray): Second binary mask.

    Returns:
        np.ndarray: Combined binary mask.
    """
    pass

Display the combined mask and any red images you detected.

In [ ]:
combined_mask = combine_masks(mask1, mask2)

show_image(combined_mask, "Combined Red Mask", cmap='gray')

masked_image = cv2.bitwise_and(original_image, original_image, mask=combined_mask)
show_image(masked_image, "Red Objects Detected")

Contours are curves that join all continuous points having the same color or intensity, which will be useful for us to identify the red object(s) in the image. There are two steps here:

1. Extract the contours.
2. From the contours, extract the largest contour.

Implement!

Hint 1: cv2.findContours

Hint 2: max() takes a key argument! See cv2.contourArea

In [ ]:
def extract_contour(mask):
    """
    Extract largest contour from a binary mask.

    Parameters:
        mask (np.ndarray): Binary mask.

    Returns:
        np.ndarray: Largest contour found in the mask.
    """
    pass

Display an outline of the contour.

In [ ]:
largest_contour = extract_contour(combined_mask)

contour_image = original_image.copy()
cv2.drawContours(contour_image, [largest_contour], -1, (0, 255, 0), 3)
show_image(contour_image, "Largest Red Contour")

Next, we want to calculate the centroid of a contour. To calculate the centroid of a contour, we need the image moments.

Image moments capture the "distribution" of pixel intensities, and are defined as weighted aaverages of the image pixels' intensities. The zero-order moment m00 is the total mass/area. The first-order moments (m10, m01) are the center of mass coordinates.

Mathematically,
- m10 is the first-order moment w.r.t to the x-axis; represents the sum of all pixel intensities multiplied by their x-coordinates.
- m01 is the first-order moment w.r.t to the y-axis; represents the sum of all pixel intensities multiplied by their y-coordinates.
- m00 is the zero-th order moment; sum of all pixel intensities ("mass")

Think about how we can use these image moments to find the centroid coordinates!

Hint: cv2.moments(). The documentation on this is a bit sparse, but it returns a dictionary with the image moments. If you wanted i.e. the m00 moment, you would just index into the dictionary with that key, i.e. M["m00"].


In [ ]:
def calculate_centroid(contour):
    """
    Calculate the centroid of a contour.

    Parameters:
        contour (np.ndarray): Contour points.

    Returns:
        tuple: Centroid coordinates (x, y).
    """
    pass

Display the centroid (in blue), with the contour.

In [ ]:
centroid = calculate_centroid(largest_contour)

centroid_image = contour_image.copy()
cv2.circle(centroid_image, centroid, 10, (255, 0, 0), -1)
cv2.putText(centroid_image, f"{centroid}", (centroid[0]-50, centroid[1]-20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
show_image(centroid_image, "Contour with Centroid")

We have finished all the helper functions. Now put it all together to create the pipeline!

In [ ]:
def detect_red_blob(image):
    pass

Putting it all together...

In [ ]:
red_blob_centroid = detect_red_blob(original_image)

result_image = original_image.copy()

hsv_image = bgr_to_hsv(original_image)
mask1, mask2 = create_red_masks(hsv_image)
mask = combine_masks(mask1, mask2)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(result_image, contours, -1, (0, 255, 0), 2)

if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(result_image, [largest_contour], -1, (255, 0, 0), 3)

    cv2.circle(result_image, red_blob_centroid, 10, (0, 0, 255), -1)
    cv2.putText(result_image, f"{red_blob_centroid}",
                (red_blob_centroid[0]-50, red_blob_centroid[1]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

show_image(result_image, "Red Blob Detection")